### Computational Geometry

in der Ebene

Die Geraden seien gegeben durch zwei Vektoren.

    Wenn der Ursprung unten links ist bedeuten positive Winkel: gegen den UhrzeigerSinn
    Wenn der Ursprung oben links ist (und das positive y also nach unten zeigt) bedeuten positive Winkel: mit dem Uhrzeigersinn
        
    Winkel werden immer im Bereich: -180 < angle <= +180 angegeben
    Winkel einer Linie: Drehung der x-Achse gegen den Uhrzeigersinn
    Winkel zwischen zwei Linien: Drehung der ersten Linie gegen Uhrzeigersinn


In [1]:
import math

class Vector:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return f'{self.x}/{self.y}'
    
    def distance(self,other):
        return math.dist((self.x,self.y),(other.x,other.y))
    
    def mag(self):
        return math.dist((self.x,self.y),(0,0))
    
    def add(self, other):
        return Vector(self.x+other.x, self.y+other.y)
    
    def multiply(self,k):
        return Vector(k*self.x, k*self.y)

    def angle(self):
        # returns winkel -180 < alpha <= 180. Positiver Winkel gegen den Uhrzeigersinn
        return math.degrees(math.atan2(self.y,self.x))
    
    def subtract(self, other):
        # returns Vector self - other
        return Vector(self.x-other.x, self.y-other.y)
    
    def vectorTo(self,other):
        # returns Vector other - self
        return Vector(other.x-self.x, other.y-self.y)
    
    def setMag(self,k):
        '''
        returns: a vector in with the same direction and magnitude k
        '''
        tmp = k/self.mag()
        return Vector(self.x*tmp, self.y*tmp)

class Line:
    def __init__(self,p1, p2):
        ''' p1 und p2 sind zwei Punkte auf der Linie '''
        self.p1 = p1
        self.p2 = p2
        
    def direction(self):
        return Vector(self.p2.x-self.p1.x,self.p2.y-self.p1.y)
    
    def angle(self):
        return self.direction().angle()
        
    def __str__(self):
        return f'{self.p1}_{self.p2}'
    
    def length(self):
        return self.p1.distance(self.p2)

    def angleWith(self, other):
        ''' angle to turn self to have same direction as other '''
        diff = other.angle()-self.angle()
        if diff > 180: diff = diff-360
        elif diff < -180: diff = 360+diff
        return diff
    
    def coordinateForm(self):
        '''
        returns: a,b,c of coordinateForm ax + by = c 
        '''
        x1, y1 = self.p1.x, self.p1.y
        x2, y2 = self.p2.x, self.p2.y
        a = -(y2-y1)
        b = (x2-x1)
        c = a*x1 + b*y1
        return a, b, c
    
    def intersection(self,other):
        '''
        returns: intersectionPoint with other line
        '''
        a,b,c = self.coordinateForm()
        a1,b1,c1 = other.coordinateForm()
        try:
            ys = (a1*c-a*c1)/(a1*b-a*b1)
            xs = (c-b*ys)/a
            return  Vector(xs, ys)
        except:
            return None
    
    def middle(self):
        return self.p1.add(self.p2).multiply(0.5)
    
    def checkLeft(self, point):
        '''
        returns: 1 if point is left of line
        0 if point is on line
        -1 if point is right of line
        '''
        p1, p2 = point.x, point.y
        a1, a2, b1, b2 = self.p1.x, self.p1.y, self.p2.x, self.p2.y
        det = (b1-a1)*(p2-a2)-(b2-a2)*(p1-a1)
        if det > 0: return 1
        elif det < 0: return -1
        return 0
    
class Triangle:
    def __init__(self,A, B, C):
        self.A = A
        self.B = B
        self.C = C
        
    def area(self):
        return abs(0.5*(self.A.x*(self.B.y-self.C.y) + self.B.x*(self.C.y-self.A.y) + self.C.x*(self.A.y-self.B.y)))
    
    def angleA(self):
        AB = Line(self.A, self.B)
        AC = Line(self.A, self.C)
        return abs(AB.angleWith(AC))
    
    def angleB(self):
        BA = Line(self.B, self.A)
        BC = Line(self.B, self.C)
        return abs(BA.angleWith(BC))
    
    def angleC(self):
        CA = Line(self.C, self.A)
        CB = Line(self.C, self.B)
        return abs(CB.angleWith(CA))
    
    def inside(self, point):
        num = 0
        AB = Line(self.A, self.B)
        BC = Line(self.B, self.C)
        CA = Line(self.C, self.A)
        orient = AB.checkLeft(point)
        return BC.checkLeft(point) and CA.checkLeft(point) == orient

In [2]:
import unittest
class MyTest(unittest.TestCase):
    def test_Vector_angle(self):
        A = Vector(3,1)
        self.assertAlmostEqual(A.angle(),18.4349,places=3)
        A = Vector(-6,1)
        self.assertAlmostEqual(A.angle(),170.5376,places=3)
        A = Vector(-3,-2)
        self.assertAlmostEqual(A.angle(),-146.3099,places=3)
        A = Vector(2,-3)
        self.assertAlmostEqual(A.angle(),-56.3099,places=3)
        
    def test_Line_angleWith(self):
        A, B = Vector(-4,-1), Vector(1,1)
        C, D = Vector(-3,3), Vector(1,-1)
        AB = Line(A,B)
        CD = Line(C,D)
        self.assertAlmostEqual(AB.angleWith(CD),-66.8014,places=3)
        self.assertAlmostEqual(CD.angleWith(AB),66.8014,places=3)
        A, B = Vector(-1,-1), Vector(-3,3)
        C, D = Vector(-4,3), Vector(1,-1)
        AB = Line(A,B)
        CD = Line(C,D)
        self.assertAlmostEqual(AB.angleWith(CD),-155.2248,places=3)
        A, B = Vector(1,1), Vector(-5,3)
        C, D = Vector(-4,3), Vector(1,-1)
        AB = Line(A,B)
        CD = Line(C,D)
        self.assertAlmostEqual(AB.angleWith(CD),159.7751,places=3)


    def test_Triangle_area(self):
        A = Vector(1,4)
        B = Vector(5,7)
        C = Vector(6,2)
        d = Triangle(A,B,C)
        self.assertEqual(d.area(),11.5)
        
    def test_Triangle_angleA(self):
        A = Vector(1,4)
        B = Vector(5,7)
        C = Vector(6,2)
        d = Triangle(A,B,C)
        self.assertAlmostEqual(d.angleA(),58.6713,places=3)
        self.assertAlmostEqual(d.angleB(),64.4400,places=3)
        self.assertAlmostEqual(d.angleC(),56.8886,places=3)
        A = Vector(1,4)
        B = Vector(-1,7)
        C = Vector(6,2)
        d = Triangle(A,B,C)
        self.assertAlmostEqual(d.angleA(),145.4914,places=3)
        self.assertAlmostEqual(d.angleB(),20.7722,places=3)
        self.assertAlmostEqual(d.angleC(),13.7362,places=3)
        
    
    def test_Linie_checkLeft(self):
        A = Vector(-5,1)
        B = Vector(-1,3)
        g = Line(A,B)
        C = Vector(-2,2)
        self.assertEqual(g.checkLeft(C),-1)
        C = Vector(5,7)
        self.assertEqual(g.checkLeft(C),1)
        C = Vector(3,5)
        self.assertEqual(g.checkLeft(C),0)
        
    def test_Triangle_inside(self):
        A = Vector(1,4)
        B = Vector(5,7)
        C = Vector(6,2)
        d = Triangle(A,B,C)
        P = Vector(4,4)
        self.assertTrue(d.inside(P))
        P = Vector(3,3)
        self.assertFalse(d.inside(P))



# Aufruf im Jupyter Notebook
unittest.main(argv=[''], verbosity=2, exit=False)

test_Line_angleWith (__main__.MyTest) ... ok
test_Linie_checkLeft (__main__.MyTest) ... ok
test_Triangle_angleA (__main__.MyTest) ... ok
test_Triangle_area (__main__.MyTest) ... ok
test_Triangle_inside (__main__.MyTest) ... ok
test_Vector_angle (__main__.MyTest) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.010s

OK


In [3]:
class Disk(Vector):
    def __init__(self,x,y,radius):
        super().__init__(x,y)
        self.radius = radius
        
    def __str__(self):
        return f'{self.x}/{self.y}-{self.radius}'
    
    def collisionDistance(self,other):
        return self.distance(other)-self.radius-other.radius
        
class Mover(Disk):
    def __init__(self,x,y,radius,vx,vy):
        super().__init(x,y,radius)
        self.vx = vx
        self.vy = vy
        

        